In [4]:
import numpy as np
import torch
import torch.nn as nn








In [ ]:
np.random.binomial(1, 0.9)

In [25]:
label = 0
idx = np.random.choice(10, 1, p=[(1/9) if i != label else 0.0 for i in range(10)])[0]


In [26]:
idx

1

In [29]:
import torchvision
import pickle
import os
import sys

import numpy as np
from torchvision.datasets import cifar


class CIFAR10Noisy(cifar.CIFAR10):
    def __init__(self, root, noise, train=True, transform=None, target_transform=None,
                 download=False):

        super(CIFAR10Noisy, self).__init__(root, transform=transform,
                                      target_transform=target_transform)

        self.train = train  # training set or test set

        if download:
            self.download()

        if not self._check_integrity():
            raise RuntimeError('Dataset not found or corrupted.' +
                               ' You can use download=True to download it')

        if self.train:
            downloaded_list = self.train_list
        else:
            downloaded_list = self.test_list
        
        c_1 = 0
        c_2 = 0
        
        self.data = []
        self.targets = []

        # now load the picked numpy arrays
        for file_name, checksum in downloaded_list:
            file_path = os.path.join(self.root, self.base_folder, file_name)
            with open(file_path, 'rb') as f:
                if sys.version_info[0] == 2:
                    entry = pickle.load(f)
                else:
                    entry = pickle.load(f, encoding='latin1')
                self.data.append(entry['data'])
                if 'labels' in entry:
                    switch = np.random.binomial(1, p=noise)
                    if switch:
                        label = int(entry['labels'])
                        new_label = np.random.choice(10, 1, p=[(1/9) if i != label else 0.0 for i in range(10)])[0]
                        assert new_label != label
                        self.targets.extend(new_label)
                        entry['labels'] = new_label
                        c_1 += 1
                    else:
                        self.targets.extend(entry['labels'])
                    c_2 += 1
                else:
                    self.targets.extend(entry['fine_labels'])
            
            if self.train:
                with open(file_path, 'wb') as f:
                    pickle.dump(entry, f)
                
        self.data = np.vstack(self.data).reshape(-1, 3, 32, 32)
        self.data = self.data.transpose((0, 2, 3, 1))  # convert to HWC

        self._load_meta()
        
        print(float(c_1) / float(c_2))

torchvision.datasets.CIFAR10(root='./data/cifar-10-noise-10', train=True,
                                            download=True)



0it [00:00, ?it/s]


 13%|█▎        | 22323200/170498071 [00:14<01:23, 1771058.14it/s]


 22%|██▏       | 38174720/170498071 [00:29<02:43, 809593.49it/s] 


 29%|██▉       | 49569792/170498071 [00:44<03:52, 520835.79it/s]


 35%|███▍      | 59580416/170498071 [01:00<02:19, 794600.34it/s]


...

 83%|████████▎ | 142336000/170498071 [02:26<00:36, 764542.81it/s]


 93%|█████████▎| 158900224/170498071 [02:40<00:07, 1529274.35it/s]


100%|█████████▉| 170237952/170498071 [02:48<00:00, 1937499.46it/s]
170500096it [02:48, 1013601.10it/s]                               


Extracting ./data/cifar-10-noise-10/cifar-10-python.tar.gz to ./data/cifar-10-noise-10


Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data/cifar-10-noise-10
    Split: Train